In [13]:
import json
from config import settings
import os


import polars as pl
from polars import col as c
from polars import selectors as cs
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp

# from distflow_algorithm import DistFlow
import time
import pandapower as pp
import pandapower.networks as pn
from polars_function import (
    get_transfo_admittance,
    get_transfo_impedance,
    get_transfo_conductance,
    get_transfo_imaginary_component,
)

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import get_all_edge_data, generate_shortest_path_length_matrix
import graphblas as gb
from utility.parser_utility import duckdb_to_changes_schema

from twindigrid_sql.entries.equipment_class import (
    TRANSFORMER,
    BRANCH,
    SWITCH,
    EXTERNAL_NETWORK,
)

from distflow_schema import DistFlowSchema

from state_estimation.matrix_generators import generate_full_jacobian_matrix

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

## Import Schema from duckdb

In [14]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
changes_schema = duckdb_to_changes_schema(file_path=file_names["duckdb_output"])
list(changes_schema.__dict__.keys())

Read and validate tables from matlab_grid.db file: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.31it/s]


['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

## Create resource data

In [15]:
resource_data = changes_schema.resource.filter(
    (c("concrete_class") == BRANCH)
    | (c("concrete_class") == SWITCH)
    | (c("concrete_class") == TRANSFORMER)
    | (c("concrete_class") == EXTERNAL_NETWORK)
).select(
    c("uuid"), c("dso_code").alias("element_id"), c("concrete_class").alias("type")
)

## Add row index to connectivity_node to give the number of node
## Join connectivity_node with connectivity to get the eq_fk
## Create the connectivity_node dictionnary with side+eq_fk as key and node index as value
cn_mapping: dict[float, str] = pl_to_dict(
    changes_schema.connectivity_node.with_row_index()
    .join(changes_schema.connectivity, left_on="uuid", right_on="cn_fk", how="left")
    .select((c("side") + c("eq_fk")).alias("eq_fk_side"), c("index"))
)

## Add node from and node to for each edge with side+eq_fk
resource_data = resource_data.with_columns(
    ("t1" + c("uuid"))
    .replace_strict(cn_mapping, default=None)
    .alias(
        "u_of_edge"
    ),  # Replace side+eq_fk with node number from connectivity for equipment
    ("t2" + c("uuid")).replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)
# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]

# Add branch parameter to line_data
resource_data = resource_data.join(
    branch, left_on="uuid", right_on="eq_fk", how="left"
).drop("uuid_right")
## Add n_tranfo to 1, useless, because automatically added by add_table from class DistFlowSchema and by default value is 1
# resource_data = resource_data.with_columns(pl.lit(1).alias("n_tranfo"))

## Search slack node id for DisFlowSchema
slack_node_id = resource_data.filter(c("type") == EXTERNAL_NETWORK)["u_of_edge"].item()

## Remove the external network from the resource_data
resource_data = resource_data.filter(c("type") != EXTERNAL_NETWORK)

## Transform resource_data in pu

In [27]:
## Transfo in pu
s_base = 1e6  # VA -> 1 MVA for distribution grid

u_b = changes_schema.base_voltage["nominal_voltage"].item()  # V
i_b = s_base / (3**0.5 * u_b)  # A
z_b = u_b**2 / s_base  # Ohm
b_b = 1 / z_b  # S


pu_base_changes_schema = {
    "U_b": u_b,
    "I_b": i_b,
    "Z_b": z_b,
    "B_b": b_b,
    "S_base": s_base,
}

resource_data_pu = resource_data.with_columns(
    (c("g") * pu_base_changes_schema["Z_b"]).alias("g_pu"),
    (c("r") / pu_base_changes_schema["Z_b"]).alias("r_pu"),
    (c("x") / pu_base_changes_schema["Z_b"]).alias("x_pu"),
    (c("b") / pu_base_changes_schema["B_b"]).alias("b_pu"),
).drop(["r", "x", "b", "g"])

## Check if B is negativ for branch and positiv for trafo

In [44]:
## If type is branch, then b_pu is negative, otherwise positive (trafo and switch)
resource_data_pu = resource_data_pu.with_columns(
    pl.when(c("type") == BRANCH)
    .then(c("b_pu").abs().neg())
    .otherwise(c("b_pu").abs())
    .alias("b_pu")
)

## Instanciation of distflowschema

In [19]:
## Create the DistFlowSchema instance
distflow_schema = DistFlowSchema()
## Add edge_data table to the DistFlowSchema instance
distflow_schema = distflow_schema.add_table(edge_data=resource_data_pu)

In [20]:
generate_full_jacobian_matrix(distflow_schema, slack_node_id)

"M_0"      nvals  nrows  ncols  dtype     format
gb.Matrix    558     58     58   FP64  csr (iso)
------------------------------------------------
     0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0   1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
1        1.0            1.0  1.0                      1.0  1.0                  
2             1.0                 1.0  1.0  1.0                 1.0  1.0        
3                  1.0                           1.0                      1.0   
4                       1.0                                                     
5                            1.0                      1.0  1.0                  
6                                 1.0                           1.0  1.0        
7                                      1.0                                      
8                                           1.0                                 
9                                          

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.03875   ,  0.        ,  0.        , ..., -0.25252625,
        -0.2331675 ,  1.        ],
       [-0.03875   ,  0.        ,  0.        , ..., -0.2331675 ,
        -0.2542975 ,  1.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

## Correct jacobian matrix

Add upflow and downflow

In [ ]:
edge_data: pt.DataFrame = distflow_schema.edge_data
    if edge_data.is_empty():
        raise ValueError("edge_data is empty")
    # Check if there is no parallel edges (nx.Graph does not support parallel edges instead of nx.MultiGraph)
    if not edge_data.filter(pl.struct("u_of_edge", "v_of_edge").is_duplicated()).is_empty():
        raise ValueError("Edges in parallel in edge_data")
    
    # check if the slack node is in the grid
    if edge_data.filter(pl.any_horizontal(c("u_of_edge", "v_of_edge")== slack_node_id)).is_empty():
        raise ValueError("The slack node is not in the grid")
    # Create nx_tree from line data
    nx_grid: nx.Graph = nx.Graph()
    _ = edge_data\
    .with_columns(
        pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_grid)
    )
    # Check if the grid is a connected tree
    if not nx.is_tree(nx_grid):
        raise ValueError("The grid is not a tree")
    elif not nx.is_connected(nx_grid):
        raise ValueError("The grid is not connected")

    nx_tree_grid : nx.DiGraph = generate_bfs_tree_with_edge_data(nx_grid, slack_node_id)
        
    # remove slack node from list of nodes
    nodes = np.array(nx_tree_grid.nodes())
    nodes = nodes[nodes != slack_node_id]

    n_nodes: int = nx_tree_grid.number_of_nodes()
    n_edges: int = nx_tree_grid.number_of_edges()
    
    descendent_matrix = generate_shortest_path_length_matrix(nx_grid = nx_tree_grid, forced_weight=1)
    print(descendent_matrix)
    # Create edge data graphblas vector
    g_pu = gb.Vector.from_dense(edge_data["g_pu"]) # type: ignore
    b_pu = gb.Vector.from_dense(edge_data["b_pu"]) # type: ignore
    n_transfo = gb.Vector.from_dense(edge_data["n_transfo"]) # type: ignore

    # Matrix values represent the sum of longitudinal resistances (or reactance) of edges in the path connecting the slack node to the 
    # lowest common ancestor of i ang j (row and column index node).
    coords, ancestor = list(zip(*nx.all_pairs_lowest_common_ancestor(nx_tree_grid)))
    x, y = list(zip(*coords))

    r_mapping = nx.shortest_path_length(nx_tree_grid, source=slack_node_id, weight="r_pu")
    r_val = list(map(lambda node_id: -2*r_mapping[node_id], ancestor))

    vn_pload_gb = gb.Matrix.from_coo(x, y, r_val, nrows=n_nodes, ncols=n_nodes).select("!=", 0)[nodes, nodes] # type: ignore
    vn_pload = (
        gb.select.offdiag(vn_pload_gb).T # type: ignore
        .ewise_add(vn_pload_gb) # type: ignore
        .to_dense(fill_value=0.0) # type: ignore
    )

    x_mapping = nx.shortest_path_length(nx_tree_grid, source=slack_node_id, weight="x_pu")
    x_val = list(map(lambda node_id: -2*x_mapping[node_id], ancestor))

    vn_qload_gb = gb.Matrix.from_coo(x, y, x_val, nrows=n_nodes, ncols=n_nodes).select("!=", 0)[nodes, nodes] # type: ignore
    vn_qload = (
        gb.select.offdiag(vn_qload_gb).T # type: ignore
        .ewise_add(vn_qload_gb) # type: ignore
        .to_dense(fill_value=0.0)
    )
    
    # Matrix value is 1 if j (column index node) is downstream i (row index node) 0 otherwise
    pflow_pload = descendent_matrix[nodes, nodes].to_dense(fill_value=0.0)
    qflow_qload = descendent_matrix[nodes, nodes].to_dense(fill_value=0.0)
    
    #TODO Check if we add or not half of upstream node
    # Matrix value is the sum of transverse susceptance (or conductance) of downstream edge (branch or transformer)
    pflow_v0 = (
        gb.select.offdiag(descendent_matrix)[nodes, nodes] # type: ignore
        .ewise_mult(g_pu)
        .reduce_rowwise(gb.monoid.plus) # type: ignore
        .to_dense(fill_value=0.0)# type: ignore
    ).reshape(-1, 1)

    qflow_v0 = (
        gb.select.offdiag(descendent_matrix)[nodes, nodes] # type: ignore
        .ewise_mult(b_pu)  # type: ignore
        .reduce_rowwise(gb.monoid.plus)  # type: ignore
        .to_dense(fill_value=0.0)
        ).reshape(-1, 1)# type: ignore
        
    # Matrix value is the multiplication of transformer ratio found in upstream edge
    # (for switch and branch, n_transfo == 1)
    vn_v0 = (
        descendent_matrix.T[nodes, nodes]  # type: ignore
        .ewise_mult(n_transfo) # type: ignore
        .reduce_rowwise(gb.monoid.times) # type: ignore
        .to_dense(fill_value=0.0)
    ).reshape(-1, 1) # type: ignore
    
    # Simple matrix
    sload_sload = np.eye(2*n_edges)
    sload_v0 = np.zeros([2*n_edges, 1]) # type: ignore
    
    pflow_qload = np.zeros([n_edges,n_edges]) # type: ignore
    qflow_pload = np.zeros([n_edges,n_edges]) # type: ignore

    v0_sload = np.zeros([1, 2*n_edges])
    v0_v0 = np.ones([1,1])
    
    # Matrix concatenation
    h = np.concatenate([
        np.concatenate([sload_sload, sload_v0], axis=1),
        np.concatenate([pflow_pload, pflow_qload, pflow_v0], axis=1),

        np.concatenate([qflow_pload, qflow_qload, qflow_v0], axis=1),

        np.concatenate([vn_pload, vn_qload, vn_v0], axis=1),
        np.concatenate([v0_sload, v0_v0], axis=1),
    ], axis=0)

## To be deleted

In [26]:
ext_grid_id = "1"
n = 0.95
v_ext_grid_sq = 1.05

line_data: pl.DataFrame = pl.DataFrame(
    {
        "line_id": np.arange(1, 13).astype(str),
        "type": [BRANCH] * 10 + [TRANSFORMER] * 2,
        "u_of_edge": ["2", "2", "2", "4", "5", "6", "7", "10", "12", "13", "8", "9"],
        "v_of_edge": ["11", "1", "3", "1", "4", "4", "8", "9", "11", "11", "4", "8"],
        "n_transfo": [1.0] * 10 + [n**2, n**2],
        "x_pu": np.arange(1, 13) * 2e-3,
        "r_pu": np.arange(1, 13) * 1e-3,
        "b_pu": list(np.arange(1, 11) * -1e-3) + [0.001, 0.001],
        "g_pu": [0.0] * 10 + [0.001, 0.001],
    }
)

node_data: pl.DataFrame = pl.DataFrame(
    {
        "node_id": np.arange(1, 14).astype(str),
        "v_base": [400.0] * 7 + [200.0, 400.0, 400.0, 200.0, 100.0, 100.0],
        "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10]) * 1e-2,
        "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1]) * 1e-2,
    },
    strict=False,
)

np.set_printoptions(linewidth=200)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq

s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

NameError: name 'DistFlow' is not defined

In [ ]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/modified_cigre_network_lv.p")
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

In [ ]:
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow: DistFlow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

In [ ]:
line_res_pp = pl.DataFrame(
    {
        "node_id": list(net.line["to_bus"].astype(str)),
        "p_pp": list(net.res_line["p_from_mw"]),
        "q_pp": list(net.res_line["q_from_mvar"]),
    }
)

node_res_pp = pl.from_pandas(net.res_bus["vm_pu"], include_index=True).select(
    c("index").cast(pl.Utf8).alias("node_id"), "vm_pu"
)

result = (
    pl.DataFrame(
        {
            "v": np.sqrt(v),
            "p": np.real(s_flow),
            "q": np.imag(s_flow),
        }
    )
    .with_row_index(name="node_id")
    .with_columns(
        c("node_id")
        .replace_strict(distflow.node_nb_to_id_mapping, default=None)
        .alias("node_id")
    )
    .join(line_res_pp, on="node_id", how="left")
    .join(node_res_pp, on="node_id", how="left")
    .with_columns(
        ((c("v") - c("vm_pu")) / c("vm_pu")).abs().alias("diff_v"),
        ((c("p") - c("p_pp")) / c("p_pp")).abs().alias("diff_p"),
        ((c("q") - c("q_pp")) / c("p_pp")).abs().alias("diff_q"),
    )
)
print(result["diff_v", "diff_p", "diff_q"].max())

In [ ]:
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

In [ ]:
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

In [ ]:
line_data = line_data.group_by("u_of_edge", "v_of_edge").agg(
    c("name", "n_transfo", "type").first(),
    c("r_pu", "x_pu").first() / c("r_pu", "x_pu").count(),
    c("g_pu", "b_pu").first() * c("g_pu", "b_pu").count(),
)